![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# TFM - Impacto de la alimentación sobre la salud de las personas

## Tabla de Contenidos
* [1. Introducción](#introduccion)
    * [1.1. Importar Librerias](#librerias)
    * [1.2. Importar datos](#data)
        * [1.2.1. Consumo de alimentos por CC.AA.](#consumos)
        * [1.2.2. Población por CC.AA.](#poblacion)
        * [1.2.3. Muertes por diabetes mellitus por CC.AA](#diabetes)


       
* [2. Extracción y limpieza de los datos](#etl)
    * [2.1. Selección de las familias de alimentos relacionadas con la diabetes](#familias_diabetes)
    * [2.2. Normalización de consumos de alimentos con la poblacion de las CC.AA.](#normalizacion_consumos)
    * [2.3. Transformación dataset de muertes por diabetes mellitus por CC.AA.](#etl_diabetes)


       
* [3. Análisis de los datos](#analisis)
    * [3.1. Clustering de CC.AA.](#clustering)

# 1. Introducción <a class="anchor" id="introduccion"></a>

## 1.1. Importar Librerías<a class="anchor" id="librerias"></a>

In [1133]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

from matplotlib import pyplot as plt
import plotly.express as px

## 1.2. Importar datos <a class="anchor" id="data"></a>

### 1.2.1. Consumos por CC.AA. <a class="anchor" id="consumos"></a>

In [1134]:
PATH_COMUNIDADES = 'data/comunidades/'

In [1135]:
# Leemos los nombres de los ficheros
paths_comunidades = [path for path in os.listdir(PATH_COMUNIDADES) if path.endswith('.xlsx')]

# Creamos una lista para ir añadiendo los dataframes leidos
dfs = list()

for path_comunidad in paths_comunidades:
    # Extraemos el nombre de la comunidad
    nombre_comunidad = os.path.splitext(path_comunidad)[0]

    # Leemos el fichero, reseteamos el index y le cambiamos el nombre de la columna a ID
    fichero = pd.read_excel('./data/comunidades/'+path_comunidad).reset_index(drop=True).rename(columns={'Unnamed: 0': 'ID'})

    # Descartamos las filas donde el la columna BIO no es nula y asignamos el nombre de la comunidad a una nueva columna.
    fichero_sin_bio = fichero[fichero['SEGMENTO BIO'].isna()].reset_index(drop=True).assign(COMUNIDAD=nombre_comunidad)

    # Añadimos el fichero leido a nuestra lista de dataframes
    dfs.append(fichero_sin_bio)

# Montamos el dataframe final
df_comunidades = pd.concat(dfs, ignore_index=True)

# Eliminamos la columna SEGMENTO BIO
df_comunidades.drop('SEGMENTO BIO', axis=1, inplace=True)

# Cambiamos el orden de la columna COMUNIDAD
cols = df_comunidades.columns.tolist()
cols = cols[:5] + cols[-1:] + cols[5:-1]
df_comunidades = df_comunidades[cols]

In [1136]:
df_comunidades.head()

,ID,SECTOR,SECCION,CATEGORIA,FAMILIA,COMUNIDAD,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,TOTAL PGC,NaN,NaN,NaN,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,2652.71,2853.52,2699.68,2703.08,2484.19,2853.67,2844.06,2412.15,2559.18,2647.53
1,ALIM. Y BEBIDAS,ALIM. Y BEBIDAS,NaN,NaN,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,1328.36,1421.87,1364.16,1405.32,1269.16,1457.10,1477.37,1288.87,1375.99,1442.49
2,ALIM.SECA,ALIM. Y BEBIDAS,ALIM.SECA,NaN,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,704.00,782.74,760.74,791.09,717.49,829.03,829.07,739.01,813.17,891.25
3,ACEITE,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,60.80,72.97,67.83,71.55,62.32,73.07,76.68,61.45,60.28,63.69
4,GIRASOL..,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,18.18,20.78,20.41,20.16,18.68,21.06,22.47,16.94,17.49,19.44


### 1.2.2. Población por CC.AA. <a class="anchor" id="poblacion"></a>

In [1137]:
PATH_POBLACION = './data/poblacion/'

In [1138]:
df_poblacion = pd.read_excel(PATH_POBLACION+'poblacion_comunidades.xlsx', skiprows=1).set_index('CC.AA.')

In [1139]:
df_poblacion

,2016,2017,2018,2019,2020
CC.AA.,,,,,
Ceuta_y_Melilla,170545,171079,171528,171264,171278
La_Rioja,315794,315381,315675,316798,313914
Pais_Vasco,2189534,2194158,2199088,2207776,2220504
Navarra,640647,643234,647554,654214,661197
Murcia,1464847,1470273,1478509,1493898,1511251
Madrid,6466996,6507184,6578079,6663394,6779888
Galicia,2718525,2708339,2701743,2699499,2701819
Extremadura,1087778,1079920,1072863,1067710,1063987
C_Valenciana,4959968,4941509,4963703,5003769,5057353


###  1.2.3. Muertes por diabetes mellitus por CC.AA <a class="anchor" id="diabetes"></a>

In [1140]:
# Guardamos el path donde tenemos los datos de las enfermedades
PATH_ENFERMEDADES = './data/enfermedades/'

In [1141]:
# Creamos el DataFrame con los datos de la tasa de muertes por comunidades
df_muertos_diabetes = pd.read_excel(PATH_ENFERMEDADES + 'diabetes.xlsx', skiprows=1).set_index('CC.AA.')

In [1142]:
# Comprobamos que se ha cargado correctamente el DataFrame
df_muertos_diabetes

,2016,2017,2018,2019,2020
CC.AA.,,,,,
Ceuta_y_Melilla,50.36,35.33,27.14,20.33,47.11
La_Rioja,15.88,15.51,19.18,15.20,15.82
Pais_Vasco,15.19,13.03,15.14,15.07,16.65
Navarra,20.41,20.21,16.13,15.77,16.34
Murcia,20.92,25.17,21.24,22.40,30.91
Madrid,10.90,10.49,10.23,8.28,10.03
Galicia,16.32,14.40,12.66,11.08,12.88
Extremadura,19.58,17.68,17.88,16.88,22.17
C_Valenciana,19.39,20.06,25.27,25.50,28.49


# 2. Extracción y limpieza de datos <a class="anchor" id="etl"></a>

## 2.1. Selección de las familias de alimentos relacionadas con la diabetes <a class="anchor" id="familias_diabetes"></a>

Nuestros datos están clasificados de la siguiente forma:

SECTOR > SECCION > CATEGORIA > FAMILIA

El objetivo es ir recorriendo cada agrupación para descartar familias de alimentos que no estén relacionadas con la diabetes.

In [1143]:
# Sacamos la lista de todos los valores de cada sector, sección, categoría y familia de alimentos
lista_sectores = list(df_comunidades[~df_comunidades['SECTOR'].isna()]['SECTOR'].unique())
lista_secciones = list(df_comunidades[~df_comunidades['SECCION'].isna()]['SECCION'].unique())
lista_categorias = list(df_comunidades[~df_comunidades['CATEGORIA'].isna()]['CATEGORIA'].unique())
lista_familias = list(df_comunidades[~df_comunidades['FAMILIA'].isna()]['FAMILIA'].unique())

### Sectores

In [1144]:
# Copiamos la primera lista para modificarla
lista_sectores_diabetes = lista_sectores.copy()

# Eliminamos los sectores que no están relacionados con la diabetes
lista_sectores_diabetes.remove('DROGUERIA Y PERFUMERIA')

# Mostramos los sectores seleccionados
lista_sectores_diabetes

['ALIM. Y BEBIDAS', 'PROD.FRESCOS']

### Secciones

In [1145]:
# Seleccionamos las secciones que hay dentro de de los sectores seleccionados en lista_sectores_diabetes
lista_secciones_sectores_diabetes = list(df_comunidades.loc[df_comunidades['SECTOR'].isin(lista_sectores_diabetes), 'SECCION'].unique())[1:]

# Copiamos la lista de los secciones de los sectores relacionados con la diabetes
lista_secciones_diabetes = lista_secciones_sectores_diabetes.copy()

# Creamos una lista de las secciones a eliminar
secciones_sin_diabetes = ['CONSERVAS', 'CHARCUTERIA', 'PL. COCINADOS Y PRECOCINADOS', 'QUESOS']

# Eliminamos las secciones que no están relacionadas con la diabetes
for i in range(len(secciones_sin_diabetes)):
    lista_secciones_diabetes.remove(secciones_sin_diabetes[i])

# Mostramos las secciones seleccionadas
lista_secciones_diabetes

['ALIM.SECA', 'BEBIDAS', 'LECHE Y BATIDOS', 'CONGELADOS', 'DERV.LACTEOS']

### Categorías

In [1146]:
# Seleccionamos las categorias que hay dentro de de los secciones seleccionados en lista_secciones_diabetes
lista_categorias_secciones_diabetes = list(df_comunidades.loc[df_comunidades['SECCION'].isin(lista_secciones_diabetes), 'CATEGORIA'].unique())[1:]

# Copiamos la lista de los categorias de las secciones relacionados con la diabetes
lista_categorias_diabetes = lista_categorias_secciones_diabetes.copy()

# Creamos una lista de las categorias a eliminar
categorias_sin_diabetes = [
    'ACEITE', 'ADITIVOS COCINA', 'ALIMENTOS INFANTILES', 'ALIMENTOS MASCOTAS', 'APERITIVOS', 'ARROZ', 'CAFES', 'DIETETICOS', 'FRUTOS SECOS',
    'INFUSIONES', 'LEGUMBRES SECAS', 'PANADERIA INDUSTRIAL', 'PASTAS', 'SOPAS Y DESHIDRATADOS', 'AGUAS', 'CERVEZA', 'ESPUMOSOS', 'VINOS',
    'LECHE LIQUIDA Y BEBIDAS VEGETALES', 'PESCADO CONGELADO', 'PESCADO PREPARADO CONGELADO', 'PLATOS PREPARADOS CONGELADOS', 'VERDURAS CONGELADAS',
    'MANTEQUILLA', 'MARGARINA', 'QUESO TIPO PETIT'
]

# Eliminamos las categorias que no están relacionadas con la diabetes
for i in range(len(categorias_sin_diabetes)):
    lista_categorias_diabetes.remove(categorias_sin_diabetes[i])

# Mostramos las categorías seleccionadas
lista_categorias_diabetes

['AZUCAR Y EDULCORANTES',
 'BOLLERIA INDUSTRIAL',
 'CACAO',
 'CEREALES DESAYUNO',
 'CHOCOLATES',
 'DULCES NAVIDENOS',
 'GALLETAS',
 'GOLOSINAS',
 'REPOSTERIA',
 'SALSAS',
 'BEBIDAS ALCOHOLICAS',
 'BEBIDAS REFRESCANTES',
 'ZUMOS',
 'BATIDOS',
 'HORCHATA.',
 'LECHE NO LIQUIDA',
 'HELADOS',
 'OTROS CONGELADOS',
 'NATA',
 'POSTRES PREPARADOS',
 'YOGURES']

In [1147]:
# Seleccionamos las familias de alimentos que hay dentro de de las categorias seleccionados en lista_categorias_diabetes
lista_familias_categorias_diabetes = list(df_comunidades.loc[df_comunidades['CATEGORIA'].isin(lista_categorias_diabetes), 'FAMILIA'].unique())[1:]

# Copiamos la lista de las familias de alimentos de las categorias relacionados con la diabetes
lista_familias_diabetes = lista_familias_categorias_diabetes.copy()

# Creamos una lista de las categorias a eliminar
famlias_sin_diabetes = [
    'EDULCORANTE', 'LEVADURA', 'L.EVAPORADA', 'L.POLVO', 'BASES PIZZA', 'CHURROS', 'FRUTAS', 'HIELO', 'HOJALDRES MASAS RELLENAS', 'HOJALDRES SIN RELLENAR',
    'NATA', 'YOGUR NATURAL'
]

# Eliminamos las familias que no están relacionadas con la diabetes
for i in range(len(famlias_sin_diabetes)):
    lista_familias_diabetes.remove(famlias_sin_diabetes[i])

# Lista de las categorias relacionadas con la diabetes. Mostramos la cantidad de familias de alimentos que hemos seleccionado
len(lista_familias_diabetes)

120

## 2.2. Normalización de consumos de alimentos con la poblacion de las CC.AA. <a class="anchor" id="normalizacion_consumos"></a>

In [1148]:
# Creamos un dataset de las familias de alimentos que tienen relación con la diabetes
df_diabetes = df_comunidades[df_comunidades['FAMILIA'].isin(lista_familias_diabetes)].groupby('COMUNIDAD').sum().T

# Editamos el índice para que tenga un formato adecuado para ser formateado a fecha (quitamos 'SEMANA', 1 para que la semana empiece en lunes, la semana y el año)
df_diabetes.index = '1 ' + df_diabetes.index.str[-5:-3] + ' ' + df_diabetes.index.str[-2:]
df_diabetes.index = pd.to_datetime(df_diabetes.index, format="%w %W %y")

# Cambiamos la granularidad a mensual
df_diabetes = df_diabetes.groupby(pd.Grouper(freq='M')).sum()

In [1149]:
# Dividimos el DF para cada año
df_diabetes_2017 = df_diabetes['2017':'2017']
df_diabetes_2018 = df_diabetes['2018':'2018']
df_diabetes_2019 = df_diabetes['2019':'2019']

In [1150]:
# Usamos la función apply para dividir los datos por la población de cada comunidad por cada 100000 habitantes
df_diabetes_2017_normalizado = df_diabetes_2017.apply(lambda x: x / df_poblacion['2017'][x.name] * 100000)
df_diabetes_2018_normalizado = df_diabetes_2018.apply(lambda x: x / df_poblacion['2018'][x.name] * 100000)
df_diabetes_2019_normalizado = df_diabetes_2019.apply(lambda x: x / df_poblacion['2019'][x.name] * 100000)

In [1151]:
# Unimos el DF con los valores ya normalizados
df_diabetes_normalizado = pd.concat([df_diabetes_2017_normalizado, df_diabetes_2018_normalizado, df_diabetes_2019_normalizado])

In [1152]:
df_diabetes_normalizado

COMUNIDAD,Andalucia,Aragon,Asturias,Baleares,C_Valenciana,Canarias,Cantabria,Castilla_La_Mancha,Castilla_y_Leon,Catalunya,Ceuta_y_Melilla,Extremadura,Galicia,La_Rioja,Madrid,Murcia,Navarra,Pais_Vasco
2017-01-31,2073.399309,2048.255205,2190.789016,2460.045215,2239.999158,3072.400493,2542.363798,1918.052316,2008.149473,2138.308035,416.357355,1859.558115,2078.109867,2106.769273,2368.821905,1817.154365,2053.187487,2339.987366
2017-02-28,1721.499388,1727.677555,1798.726521,2083.448103,1854.717253,2570.428832,2106.256301,1574.079771,1651.574882,1791.590335,315.485828,1576.475109,1703.776004,1690.945238,1926.031137,1495.416157,1672.801811,1883.287348
2017-03-31,1739.886537,1731.700478,1818.183311,2196.094262,1911.025154,2494.888102,2127.407612,1582.799035,1679.744958,1814.469489,388.966501,1580.281873,1726.973248,1740.954591,1897.038412,1514.821397,1688.130603,1881.800217
2017-04-30,1833.163362,1808.957402,1874.562302,2429.778163,2074.894126,2591.175744,2353.759726,1658.656083,1800.838568,1910.353065,638.699081,1877.492777,1841.022117,1904.087437,1849.476978,1657.236445,1728.613537,1790.515542
2017-05-31,2273.424131,2255.265712,2228.732511,3178.234927,2449.898806,3045.375479,2694.174515,2027.196934,2135.179267,2332.146568,875.782533,2153.145603,2198.197124,2282.036648,2398.445472,1972.722753,2182.407957,2348.168637
2017-06-30,2028.745725,1909.110220,1869.822022,2959.906774,2305.712688,2585.348279,2358.093728,1791.439144,1892.185303,2097.956148,755.721041,1790.315949,1910.456926,1935.563652,2015.677596,1817.837912,1827.058271,1904.958531
2017-07-31,2742.904979,2413.135435,2467.001623,4326.095274,3391.402302,3467.607410,3621.940565,2341.258758,2565.376550,2847.195874,846.217245,2354.403104,2543.767601,2602.702763,2215.021582,2693.470532,2282.506833,2251.243529
2017-08-31,2167.980696,1896.421777,1946.624990,3321.090790,2588.819731,2794.178323,2972.484685,1878.889223,2137.643607,2171.479639,675.413113,2030.461516,2034.472051,2075.695746,1624.020928,2067.776529,1792.482984,1731.577671
2017-09-30,1900.162295,1765.351671,1779.851395,2735.345641,2112.590709,2638.397891,2225.742079,1700.887383,1754.983199,1895.149838,638.915355,1762.719461,1793.725601,1844.391387,1925.662007,1684.534777,1726.415270,1848.559675
2017-10-31,2352.409240,2340.783190,2289.268184,3187.208949,2561.253860,3458.246941,2793.374060,2095.389123,2228.159276,2434.846602,871.644094,2123.487851,2279.680276,2326.325936,2538.427836,2038.723421,2216.171720,2425.418315


In [1153]:
fig = px.line(
    df_diabetes_normalizado,
    title='Consumo normalizado de alimentos relacionados con la diabetes por CCAA',
    labels = {
        'index' : 'Tiempo',
        'value' : 'Consumo / Población * 100'
    }
)
fig.show()

## 2.3. Transformación dataset de muertes por diabetes mellitus por CC.AA. <a class="anchor" id="etl_diabetes"></a>

Los datos de diabetes mellitus que tenemos se refieren a la *** Tasa de mortalidad ajustada por diabetes mellitus, por 100 000 hab. ***

Por lo tanto para obtener los datos aproximados de muertes tenemos que dividir la población de cada CC.AA entre 100.000 y multiplicar por la tasa.

El objetivo de este punto es transformar el dataset para que tenga una granularidad similar a la de los datos de consumos.

In [1154]:
#df_aux = df_muertos_diabetes.set_index('CC.AA.').T
fig = px.line(
    df_muertos_diabetes.T,
    title='Tasa ajustada de muertes por diabetes mellitus por CC.AA.',
    labels = {
        'index' : 'Tiempo',
        'value' : 'Tasa de muertes (*Poblacion / 100.000)'
    }
)
fig.show()

In [1160]:
df_muertos_diabetes

CC.AA.,Ceuta_y_Melilla,La_Rioja,Pais_Vasco,Navarra,Murcia,Madrid,Galicia,Extremadura,C_Valenciana,Catalunya,Castilla_La_Mancha,Castilla_y_Leon,Cantabria,Canarias,Baleares,Asturias,Aragon,Andalucia
2017-01-01,35.33,15.51,13.03,20.21,25.17,10.49,14.40,17.68,20.06,20.81,23.78,18.90,9.64,24.13,28.97,16.32,23.98,21.62
2018-01-01,27.14,19.18,15.14,16.13,21.24,10.23,12.66,17.88,25.27,19.91,18.72,19.46,13.06,23.11,24.07,19.93,20.84,20.95
2019-01-01,20.33,15.20,15.07,15.77,22.40,8.28,11.08,16.88,25.50,18.47,17.36,19.52,14.83,19.31,24.66,18.04,21.21,20.82
2020-01-01,47.11,15.82,16.65,16.34,30.91,10.03,12.88,22.17,28.49,21.28,21.38,25.20,18.95,20.00,22.35,21.08,20.91,23.66


In [1159]:
# Eliminamos el año 2016
df_muertos_diabetes.drop(['2016'], axis=1, inplace=True)

# Transponemos la tabla
df_muertos_diabetes = df_muertos_diabetes.T

# Cambiamos el index a Datetime
df_muertos_diabetes.index = pd.to_datetime(df_muertos_diabetes.index, format="%Y")

# Cambiamos la granularida a mensual con un backfill
df_muertos_diabetes = df_muertos_diabetes.resample('M').bfill()

# Eliminamos las 3 ultimas filas para que el último dato sea de Octubre de 2019, igual que los datos de consumos
df_muertos_diabetes.drop(df_muertos_diabetes.tail(3).index , axis=0, inplace=True)

# Mostramos el resultado de nuestro dataset
df_muertos_diabetes.head()

# 3. Análisis de los datos <a class="anchor" id="analisis"></a>

In [ ]:
df_diabetes_normalizado.shape, df_muertos_diabetes.shape

((34, 18), (34, 18))

In [ ]:
df_aux= df_diabetes_normalizado / df_muertos_diabetes

In [ ]:
fig = px.line(
    df_aux
)
fig.show()

In [ ]:
df_aux

,Andalucia,Aragon,Asturias,Baleares,C_Valenciana,Canarias,Cantabria,Castilla_La_Mancha,Castilla_y_Leon,Catalunya,Ceuta_y_Melilla,Extremadura,Galicia,La_Rioja,Madrid,Murcia,Navarra,Pais_Vasco
2017-01-31,98.968941,98.284799,109.924185,102.203790,88.642626,132.946798,194.667978,102.460060,103.193704,107.398696,15.341096,104.002132,164.147699,109.841985,231.556393,85.553407,127.289987,154.556629
2017-02-28,82.171808,82.901994,90.252209,86.557877,73.396013,111.225826,161.275368,84.085458,84.870241,89.984447,11.624386,88.169749,134.579463,88.161900,188.272838,70.405657,103.707490,124.391502
2017-03-31,83.049477,83.095033,91.228465,91.237817,75.624264,107.957079,162.894917,84.551230,86.317829,91.133576,14.331853,88.382655,136.411789,90.769270,185.438750,71.319275,104.657818,124.293277
2017-04-30,87.501831,86.802179,94.057316,100.946330,82.108988,112.123572,180.226625,88.603423,92.540523,95.949426,23.533496,105.005189,145.420388,99.274632,180.789538,78.024315,107.167609,118.263906
2017-05-31,108.516665,108.218124,111.828024,132.041335,96.948904,131.777390,206.292076,108.290435,109.721442,117.134433,32.269069,120.422014,173.633264,118.980013,234.452148,92.877719,135.301175,155.097004
2017-06-30,96.837505,91.607976,93.819469,122.970784,91.243082,111.871410,180.558478,95.696535,97.234599,105.371981,27.845285,100.129527,150.904970,100.915727,197.035933,85.585589,113.270817,125.822888
2017-07-31,130.926252,115.793447,123.783323,179.729758,134.206660,150.047919,277.330824,125.067241,131.828189,143.003309,31.179707,131.678026,200.929510,135.698788,216.522149,126.811230,141.506933,148.695081
2017-08-31,103.483565,90.999126,97.673105,137.976352,102.446368,120.907760,227.602196,100.368014,109.848078,109.064773,24.886261,113.560487,160.700794,108.221885,158.750824,97.352944,111.127277,114.371048
2017-09-30,90.699871,84.709773,89.305138,113.641281,83.600740,114.166936,170.424355,90.859369,90.184132,95.185828,23.541465,98.586100,141.684487,96.162220,188.236755,79.309547,107.031325,122.097733
2017-10-31,112.286837,112.321650,114.865438,132.414165,101.355515,149.642879,213.887753,111.933180,114.499449,122.292647,32.116584,118.763303,180.069532,121.289152,248.135663,95.985095,137.394403,160.199360
